In [1]:
#Imports moduls
%run "C:\Users\Илья\Desktop\Work\support_functions.ipynb"

Automatically created module for IPython interactive environment


### Ноутбук для расчета статистик

1. Пишу функцию которая принимает номер кластера (или "all_clusters"), проверяет что в кластере больше 50 значений, и выводит корреляционные таблицы для него, и сохраняет их в html в папке Corr matrixs в соответствующей директории
2. Скорее всего параметры придется настраивать индивидуально под кластеры, разом запустить вероятно не получится, копирую отдельную функцию для каждого кластера (или пробую черех @iteract
3. Считаю таблицу показателей (с 95% DI для каждого кластера). Делаю функцию которая считает все средние ЦП (интервал), возраст, % мучин,  топ 3 наиболее частых диагнозов, топ 3 наиболее частых вмешательств для каждого кластера. Таблицу на первый план, далее описание кластеров и корреляции, в финале таблица кластера и ЦП, и топ-1 фичи наиболее сильно увеличивающей эффективность, и тоже для фичи снижающей эффективность терапии (Therapi's plan's claster). Лучше конешно автоматизировать - делаю функцию которая выдаст готовую таблицу.

логика - анализ планов лечения анализ факторов прогресса лечения для каждого из планов

In [2]:
#Extract data
# df=pd.read_pickle("Data/mental_pats_for_modeling_T0_Last_Points.pkl")
# df=pd.read_pickle("Data/mental_pats_for_modeling_T0_Last_Points_old.pkl")
df=pd.read_pickle("Data/mental_pats_for_modeling_T0_Last_Points_simple_with_clustering.pkl")

In [3]:
df.head()

,T0_measurement_point_ID,T0_client_ID,T0_measurement_point,T0_measurement_point_status,T0_status_date,T0_case_manager,T0_age,T0_WQO_quality_of_life,T0_WQO_health_status,T0_WQOLBR03,T0_WQOLBR04,T0_WQOLBR05,T0_WQOLBR06,T0_WQOLBR07,T0_WQOLBR08,T0_WQOLBR09,T0_WQOLBR10,T0_WQOLBR11,T0_WQOLBR12,T0_WQOLBR13,T0_WQOLBR14,T0_WQOLBR15,T0_WQOLBR16,T0_WQOLBR17,T0_WQOLBR18,T0_WQOLBR19,T0_WQOLBR20,T0_WQOLBR21,T0_WQOLBR22,T0_WQOLBR23,T0_WQOLBR24,T0_WQOLBR25,T0_WQOLBR26,T0_WQOLBRX1,T0_WQOLBRX2,T0_WQOLBRX3,T0_BSI1801,T0_BSI1802,T0_BSI1803,T0_BSI1804,T0_BSI1805,T0_BSI1806,T0_BSI1807,T0_BSI1808,T0_BSI1809,T0_BSI1810,T0_BSI1811,T0_BSI1812,T0_BSI1813,T0_BSI1814,T0_BSI1815,T0_BSI1816,T0_BSI1817,T0_BSI1818,T0_ICF2_01,T0_ICF2_02,T0_ICF2_03,T0_ICF2_04,T0_ICF2_05,T0_ICF2_06,T0_ICF2_07,T0_ICF2_08,T0_ICF2_09,T0_ICF2_10,T0_ICF2_11,T0_ICF2_12,T0_ICF2_13,T0_a5_5,T0_a5_4,T0_a5_3,T0_a5_2,T0_a5_1,T0_a4_5,T0_a4_4,T0_a4_3,T0_a4_2,T0_a4_1,T0_a3_5,T0_a3_4,T0_a3_3,T0_a3_2,T0_a3_1,T0_a2_5,T0_a2_4,T0_a2_3,T0_a2_2,T0_a2_1,T0_a1_5,T0_a1_4,T0_a1_3,T0_a1_2,T0_a1_1,T0_Massnahme_1,T0_Massnahme_2,T0_Massnahme_3,T0_Massnahme_4,T0_Massnahme_5,T0_Massnahme_6,T0_Massnahme_7,T0_Massnahme_8,T0_Massnahme_9,T0_Massnahme_10,T0_Massnahme_11,T0_Massnahme_12,T0_Massnahme_13,T0_Massnahme_14,T0_Massnahme_15,T0_all_massnahme,T0_types_massnahme,T0_group,T0_all_diagnosis,T0_GAF,T0_BSI18_Somatization,T0_BSI18_Depression,T0_BSI18_Anxiety,T0_ICF13_mean,T0_WQO_physical_health,T0_WQO_psychological,T0_WQO_social_relationships,T0_WQO_environment,Last_Point_measurement_point_ID,Last_Point_client_ID,Last_Point_measurement_point,Last_Point_measurement_point_status,Last_Point_status_date,Last_Point_organization,Last_Point_case_manager,Last_Point_age,Last_Point_gender,Last_Point_WQO_quality_of_life,Last_Point_WQO_health_status,Last_Point_WQOLBR03,Last_Point_WQOLBR04,Last_Point_WQOLBR05,Last_Point_WQOLBR06,Last_Point_WQOLBR07,Last_Point_WQOLBR08,Last_Point_WQOLBR09,Last_Point_WQOLBR10,Last_Point_WQOLBR11,Last_Point_WQOLBR12,Last_Point_WQOLBR13,Last_Point_WQOLBR14,Last_Point_WQOLBR15,Last_Point_WQOLBR16,Last_Point_WQOLBR17,Last_Point_WQOLBR18,Last_Point_WQOLBR19,Last_Point_WQOLBR20,Last_Point_WQOLBR21,Last_Point_WQOLBR22,Last_Point_WQOLBR23,Last_Point_WQOLBR24,Last_Point_WQOLBR25,Last_Point_WQOLBR26,Last_Point_WQOLBRX1,Last_Point_WQOLBRX2,Last_Point_WQOLBRX3,Last_Point_BSI1801,Last_Point_BSI1802,Last_Point_BSI1803,Last_Point_BSI1804,Last_Point_BSI1805,Last_Point_BSI1806,Last_Point_BSI1807,Last_Point_BSI1808,Last_Point_BSI1809,Last_Point_BSI1810,Last_Point_BSI1811,Last_Point_BSI1812,Last_Point_BSI1813,Last_Point_BSI1814,Last_Point_BSI1815,Last_Point_BSI1816,Last_Point_BSI1817,Last_Point_BSI1818,Last_Point_ICF2_01,Last_Point_ICF2_02,Last_Point_ICF2_03,Last_Point_ICF2_04,Last_Point_ICF2_05,Last_Point_ICF2_06,Last_Point_ICF2_07,Last_Point_ICF2_08,Last_Point_ICF2_09,Last_Point_ICF2_10,Last_Point_ICF2_11,Last_Point_ICF2_12,Last_Point_ICF2_13,Last_Point_a5_5,Last_Point_a5_4,Last_Point_a5_3,Last_Point_a5_2,Last_Point_a5_1,Last_Point_a4_5,Last_Point_a4_4,Last_Point_a4_3,Last_Point_a4_2,Last_Point_a4_1,Last_Point_a3_5,Last_Point_a3_4,Last_Point_a3_3,Last_Point_a3_2,Last_Point_a3_1,Last_Point_a2_5,Last_Point_a2_4,Last_Point_a2_3,Last_Point_a2_2,Last_Point_a2_1,Last_Point_a1_5,Last_Point_a1_4,Last_Point_a1_3,Last_Point_a1_2,Last_Point_a1_1,Last_Point_Massnahme_1,Last_Point_Massnahme_2,Last_Point_Massnahme_3,Last_Point_Massnahme_4,Last_Point_Massnahme_5,Last_Point_Massnahme_6,Last_Point_Massnahme_7,Last_Point_Massnahme_8,Last_Point_Massnahme_9,Last_Point_Massnahme_10,Last_Point_Massnahme_11,Last_Point_Massnahme_12,Last_Point_Massnahme_13,Last_Point_Massnahme_14,Last_Point_Massnahme_15,Last_Point_referrer,Last_Point_all_massnahme,Last_Point_types_massnahme,Last_Point_group,Last_Point_all_diagnosis,Last_Point_GAF,Last_Point_BSI18_Somatization,Last_Point_BSI18_Depression,Last_Point_BSI18_Anxiety,Last_Point_ICF13_mean,Last_Point_WQO_physical_health,Last_Point_WQO_psychological,Last_Point_WQO_social_relationships,Last_Point

In [4]:
#Смотрим количество непустых величин в прогрессе, отбираем целевые признаки с заполненностью 95 значений минимум
def selected_goal_features(df,border=50):
    goal_features=[]
    for c in df.columns:
        if "Progress" in c:
            print(c,df[c].value_counts().sum())
            if (df[c].value_counts().sum()>border)&(c!='Progress_WQOLBRX2'):
                goal_features.append(c)
    #WQOLBRX2 Time needed to complete the questionnaire in minutes 
    #Не совсем целевая переменная, ее не берем

#     print("Отобранные целевые переменные",goal_features)
    return goal_features

In [5]:
rename_dict={'Client:Treatment/Care:Life Skills:Activity Development/Maintenance:Group:N/A':'Life Skills - activity development  Group - NA',
 'Client:treatment/care:everyday skills:activity development/maintenance:group:mobile':'Everyday skills - activity development  Group - mobile',
 'Client:Treatment/care:Relaxation methods:Relaxation according to Jacobson:Group:Outpatient':'Relaxation methods - Relaxation according to Jacobson  Group - Outpatient',
 'Client:Treatment/Care:Social Competence:Communication Training:Group:N/A':'Social Competence - Communication Training  Group - NA',
 'Client:Treatment/care:Therapy:Resource work:Group:mobile':"Therapy - Resource work  Group - mobile",
 'Client:Treatment/Care:Therapy:Cognitive Training:Group:N/A':'Therapy - Cognitive Training  Group - NA',
 'Client: Treatment/care: Relaxation techniques: Jacobson relaxation: Group: N/A':"Relaxation techniques - Jacobson relaxation  Group - NA",
 'Client:Treatment/Care:Therapy:Resource Work:Group:N/A':'Therapy - Resource Work Group - NA',
 'Client: Treatment/care: Therapy: Dealing with psychotic symptoms: Group: Outpatient':"Therapy - Dealing with psychotic symptoms  Group - Outpatient",
 'Client: Treatment/care: Therapy: Resource work: Group: Outpatient':'Therapy - Resource work  Group - Outpatient',
 'Client:Treatment/care:Everyday competence:Building/maintaining daily structure:Group:N/A':"Everyday competence - Building daily structure  Group - NA",
 'Client:Treatment/Care:Life Skills:Training in Health Awareness and Behavior:Group:N/A':'Life Skills - Training in Health Awareness and Behavior  Group - NA',
 'Client:Treatment/Care:Therapy:Depression Coping:Group:N/A':'Therapy - Depression Coping - Group - NA',
 'Client: Treatment/Care: De-escalation (crisis intervention): N/A: Group: N/A':'De-escalation (crisis intervention)  Group - NA',
 'Client:Treatment/Care:Psychoeducation:N/A:Group:N/A':"Psychoeducation  Group - NA",
 'Client:treatment/care:everyday skills:activity development/maintenance:group:outpatient':"Everyday skills - activity development  Group - Outpatient",
 'Client:Treatment/care:N/A:Therapy in an individual setting:Individual:N/A':"Therapy in an individual setting  Individual - NA",
 'Client:Treatment/Care:Therapy:Emotional Competence Traning:Group:N/A':'Therapy - Emotional Competence Traning  Group - NA',
 'duration_treatment':'treatment duration (months)',
 "T0_age":"age (years)",
 }

def remane_col(col):
    if col in rename_dict:
        col=rename_dict[col]
    col=col.replace("T0_","")
    
#Поднимаем только 1 букву если она маленькая
#     if col[0].islower():
#         col=col.capitalize()

    if (len(col)==3)&(col!="Age"):
        col=f"ICD {col}"
        
    #Чистим целевые переменные 
    col=col.replace("Progress_","").replace("_"," ")
    return col

In [11]:
def calculate_pvalues(df):
#     df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            data_for_corr=df.dropna(subset=[r,c],how='any',axis=0)
            print(data_for_corr.shape[0])
            
            if data_for_corr.shape[0]>10:
                pvalues[r][c] = round(pearsonr(data_for_corr[r], data_for_corr[c])[1], 4)
            else:
                pvalues[r][c] = 1
#             try:
#                 print(data_for_corr.shape)
            
#             except:
#             pvalues[r][c] = 1
#             print(f"Не посчиталось p-value для признаков {r} и {c}, Размерность данных {data_for_corr.shape}")
    return pvalues

def corr_matrix_plotly(data,method_corr="pearson",title='Heatmap',
                       x_features=None,
                       y_features=None,
                       file_name="corr_matrix",
                       width=1500,
                       height=750,
                       x_border=None,
                       y_border=None,
                       rename_cols_function=None,
                       clusters=None,
                       common_featires_for_clust=None,
                       numbering=None, #columns,index
                       alpha=None
                      ):
    #Если нужно посчитать корреляции для различных кластеров генерирую новый датасет
    if (clusters is not None)&(x_features is not None)&(y_features is not None):
        df_mask=pd.DataFrame(index=y_features)
        
        for clust in tqdm_notebook(clusters):
            data_clust=data[data.cluster==clust]
            corr =data_clust.corr(method=method_corr).round(2)
            mask = np.triu(np.ones_like(corr, dtype=bool))
            data_clust_mask = corr.mask(mask)
            
            
            #Применяем alpha убирая корреляции для которых pvalue>alpha, делая значения в этих матрицах пустыми
#             if alpha is not None:
            pvalues=calculate_pvalues(data_clust)
            print(pvalues)
            for c,r in zip(data_clust_mask.columns,data_clust_mask.columns):
                if pvalues[r][c]>alpha:
                    print(pvalues[r][c],data_clust_mask[r][c])
                    data_clust_mask[r][c]=None
                
            data_clust_mask=data_clust_mask[x_features]
            
            #Переименовываем признаки
            data_clust_mask.columns=[f"Cluster #{clust} {c}" for c in data_clust_mask.columns]
            df_mask=df_mask.join(data_clust_mask)
        
        #Сортируем колонки для красивого отображения
        old_columns=df_mask.columns.tolist()
        old_columns.sort(key=lambda line:re.sub(r"Cluster #[0-9]","",line))
        print(old_columns)
        df_mask.columns=old_columns
        
    else:
        corr = data.corr(method=method_corr).round(2)
        mask = np.triu(np.ones_like(corr, dtype=bool))
        df_mask = corr.mask(mask)
        

    
    
        #Выбираем какие колонки показывать по x какие по y
        if x_features is not None:
            df_mask=df_mask[x_features]

        if y_features is not None:
            df_mask=df_mask.loc[y_features]
    
    #Применяем границы x_border и y_border - удаляем из матрицы корреляции все строки (столбцы) в которых связей > border
    if x_border is not None:
        #Фильтруем столбцы
        df_mask=df_mask.loc[:,list(filter(lambda f:df_mask[f].apply(abs).max()>=x_border,df_mask.columns.tolist()))]
        
    if y_border is not None:
        #Фильтруем строки
        df_mask=df_mask.loc[list(filter(lambda f:df_mask.loc[f].apply(abs).max()>=y_border,df_mask.index.tolist()))]
        
    
    #Переименовываем колонки и индексы если необходимо
    if rename_cols_function is not None:
        df_mask.columns=[remane_col(c) for c in df_mask.columns]
        df_mask.index=[remane_col(i) for i in df_mask.index]
    
    #Нумеруем строки/столбцы если это необходимо
    if numbering=="columns":
        df_mask.columns=[f"{ind}. {c}" for ind,c in enumerate(df_mask.columns.tolist())]
    if numbering=="index":
        df_mask.index=[f"{ind}. {c}" for ind,c in enumerate(df_mask.index.tolist())]
        print(df_mask.index.tolist())
    
    fig = ff.create_annotated_heatmap(z=df_mask.to_numpy(), 
                                      x=df_mask.columns.to_list(),
                                      y=df_mask.index.to_list(),
                                      colorscale=px.colors.sequential.Blackbody, #Turbo Blackbody(лучшее) Jet(оч неплохо) RdBu(неплохо не идеал)
                                      #Настройка палитры тут https://plotly.com/python/builtin-colorscales/
                                      hoverinfo="none", #Shows hoverinfo for null values
                                      showscale=True, ygap=1, xgap=1
                                     )
    
   
        
    fig.update_xaxes(side="bottom")

    fig.update_layout(
        title_text=title, 
        title_x=0.5, 
        width=width, 
        height=height,
        xaxis_showgrid=False,
        yaxis_showgrid=False,
        xaxis_zeroline=False,
        yaxis_zeroline=False,
        yaxis_autorange='reversed',
        template='plotly_white'
    )

    # NaN values are not handled automatically and are displayed in the figure
    # So we need to get rid of the text manually
    for i in range(len(fig.layout.annotations)):
        if fig.layout.annotations[i].text == 'nan':
            fig.layout.annotations[i].text = ""

    fig.show()
    
    
    # html file
    import plotly
    plotly.offline.plot(fig, filename=f'Data/{file_name}.html')

In [12]:
#Вмешательства
interventions=[c for c in df.columns if (':' in c)&(" : " not in c)&("Client" in c)]

In [13]:
from ipywidgets import interact

@interact
def create_corr_matrix_graph(cluster=list(range(8)),for_all_patients=[False,True],method_corr=['pearson', 
                                                                                             "kendall",
                                                                                             "spearman"]):
    #Выбираем способ построения таблицы
    if for_all_patients==True:
        df_for_graph=df.copy()
        cluster="all_patients"
    else:
        df_for_graph=df[df.cluster==cluster]
        
    goal_features=selected_goal_features(df_for_graph,90)

    #Вмешательства
    interventions=[c for c in df_for_graph.columns if (':' in c)&(" : " not in c)&("Client" in c)]

    #Диагнозы
    diagnosis=[c for c in df_for_graph.columns if (re.match(r"[A-Z][0-9][0-9]",c)!=None)]

    #Социальные признаки
    social_features=["T0_age",'T0_gender : women','T0_gender : man']+[c for c in df_for_graph.columns if "T0_referrer" in c]+["duration_treatment"]
    df_for_graph["T0_age"]=df_for_graph["T0_age"].astype(int)

    factors=diagnosis+social_features+interventions + ['cluster']
    
   
    
    corr_matrix_plotly(df_for_graph[goal_features+factors],
                       method_corr=method_corr,
                       y_features=interventions,
                       x_features=goal_features,
                       title=f"Correlations for patients from Therapy Plan's Cluster #{cluster}",
#                        file_name=f"Correlation matrices/correlation_matrix_cluster_{cluster}_{method_corr}_test",
                       file_name=f"Correlation matrices/correlation_matrix_cluster_{cluster}_{method_corr}_test",
                       x_border=0.3,
                       y_border=0.3,
                       width=2050,
                       height=750, #750
                       rename_cols_function=remane_col,
                       clusters=range(7),
                       numbering="index",
                       alpha=0.000000000000000000001
                      )

interactive(children=(Dropdown(description='cluster', options=(0, 1, 2, 3, 4, 5, 6, 7), value=0), Dropdown(des…

### Cчитаю статистики для кластера

Считаю таблицу показателей (с 95% DI для каждого кластера). Делаю функцию которая считает все средние ЦП (интервал), возраст, % мучин,  топ 3 наиболее частых диагнозов, топ 3 наиболее частых вмешательств для каждого кластера. 

In [9]:
df.columns.to_list()

['T0_measurement_point_ID',
 'T0_client_ID',
 'T0_measurement_point',
 'T0_measurement_point_status',
 'T0_status_date',
 'T0_case_manager',
 'T0_age',
 'T0_WQO_quality_of_life',
 'T0_WQO_health_status',
 'T0_WQOLBR03',
 'T0_WQOLBR04',
 'T0_WQOLBR05',
 'T0_WQOLBR06',
 'T0_WQOLBR07',
 'T0_WQOLBR08',
 'T0_WQOLBR09',
 'T0_WQOLBR10',
 'T0_WQOLBR11',
 'T0_WQOLBR12',
 'T0_WQOLBR13',
 'T0_WQOLBR14',
 'T0_WQOLBR15',
 'T0_WQOLBR16',
 'T0_WQOLBR17',
 'T0_WQOLBR18',
 'T0_WQOLBR19',
 'T0_WQOLBR20',
 'T0_WQOLBR21',
 'T0_WQOLBR22',
 'T0_WQOLBR23',
 'T0_WQOLBR24',
 'T0_WQOLBR25',
 'T0_WQOLBR26',
 'T0_WQOLBRX1',
 'T0_WQOLBRX2',
 'T0_WQOLBRX3',
 'T0_BSI1801',
 'T0_BSI1802',
 'T0_BSI1803',
 'T0_BSI1804',
 'T0_BSI1805',
 'T0_BSI1806',
 'T0_BSI1807',
 'T0_BSI1808',
 'T0_BSI1809',
 'T0_BSI1810',
 'T0_BSI1811',
 'T0_BSI1812',
 'T0_BSI1813',
 'T0_BSI1814',
 'T0_BSI1815',
 'T0_BSI1816',
 'T0_BSI1817',
 'T0_BSI1818',
 'T0_ICF2_01',
 'T0_ICF2_02',
 'T0_ICF2_03',
 'T0_ICF2_04',
 'T0_ICF2_05',
 'T0_ICF2_06',
 'T0

In [10]:
#Возвращяет топ-3 наиболее частых событий через запятую
def three_most_frequent_events(df,type_events):
    if type_events=="interventions":
        events=[c for c in df.columns if (':' in c)&(" : " not in c)&("Client" in c)]
    if type_events=="diagnoses":
        events=[c for c in df.columns if (re.match(r"[A-Z][0-9][0-9]",c)!=None)]
    
    intervs_stats=df.describe()[events].T
    intervs_stats.sort_values("mean",inplace=True)
    
    #Удаляем вмешательства с нулевой частотой
    intervs_stats=intervs_stats[intervs_stats['mean']>0]
    
    # intervs_stats
    most_frequent_values=intervs_stats[-3:].index.tolist()
    most_frequent_values=[remane_col(event) for event in most_frequent_values]
    most_frequent_values.reverse()
    
    #Если топовых вмешательств меньше 3 добавляем None
    while len(most_frequent_values)<3:
        most_frequent_values.append(None)
    
    return " | ".join(most_frequent_values)

In [ ]:
#Вытаскиваем вмешательство, частота которого больше, чем частота для всех остальных кластеров (если такого нет - None) 
#Если неcколько - вытаскиваем все, и частоты
def get_specific_interventions(df,cluster,type_events="interventions"):
    if type_events=="interventions":
        events=[c for c in df.columns if (':' in c)&(" : " not in c)&("Client" in c)]
    elif type_events=="diagnoses":
        events=[c for c in df.columns if (re.match(r"[A-Z][0-9][0-9]",c)!=None)]
    else:
        events=[]
    events_data=df.groupby("cluster").agg(np.mean)[events]
    
    specific_interventions=[]
    for ev in events:
        
        if events_data.loc[cluster,ev]==events_data[ev].max():
            frequency_ev=int(events_data.loc[cluster,ev]*100)
            
            #Имеет смысл добавлять диагнозы с частотой хотябы 15%
            if frequency_ev>10:
                line_ev=f"{frequency_ev}% - {remane_col(ev)}"
                specific_interventions.append((line_ev,frequency_ev))
    specific_interventions.sort(key=lambda Tuple:Tuple[1])
    interventions=[line_ev for line_ev,frequency_ev in specific_interventions]
    return " | ".join(interventions)

#Рисуем слолбчатую диаграмму, которая возвращает частоты для набора событий
def show_frequency_events_graph(df,type_events="interventions"):
    if type_events=="interventions":
        events=[c for c in df.columns if (':' in c)&(" : " not in c)&("Client" in c)]
    elif type_events=="diagnoses":
        events=[c for c in df.columns if (re.match(r"[A-Z][0-9][0-9]",c)!=None)]
    else:
        events=[]
    events_data=df.groupby("cluster").agg(np.mean)[events]
    #Переводим в проценты
    events_data=events_data*100
    
    
    return " | ".join(interventions)

In [ ]:
events=[c for c in df.columns if (':' in c)&(" : " not in c)&("Client" in c)&("Therapy in an individual" not in c)]
events_data=df.groupby("cluster").agg(np.mean)[events]
# events_data=events_data*100

#Делаем красивый порядок колонок
cols=events_data.columns.tolist()
cols.sort()
events_data=events_data[cols]
events_data=events_data.T
events_data

In [ ]:
# table_data = [['Team', 'Wins', 'Losses', 'Ties'],
#                   ['Montréal<br>Canadiens', 18, 4, 0],
#                   ['Dallas Stars', 18, 5, 0],
#                   ['NY Rangers', 16, 5, 0],
#                   ['Boston<br>Bruins', 13, 8, 0],
#                   ['Chicago<br>Blackhawks', 13, 8, 0],
#                   ['Ottawa<br>Senators', 12, 5, 0]]

def Vertical_Table_and_Chart(events_data):
    #Add table data
    
    # Initialize a figure with ff.create_table(table_data)
    fig = go.Figure()
#     fig = ff.create_table()
    clusters=events_data.index.tolist()

    events=events_data.columns.tolist()
    colors_names=["green", "pink", "aqua", "blue", "yellow","black", "red", "coral", "blanchedalmond", "blue",
 "blueviolet", "brown", "burlywood", "cadetblue","chartreuse", "chocolate", "coral", "cornflowerblue"]
    
    neq_events_names=[remane_col(ev) for ev in events_data.index.tolist()]
    traces=[]
    for i in range(len(events)):
        event=events[i]
        color=colors_names[i]
        
        # Make traces for graph
        trace = go.Bar(x=neq_events_names, y=events_data[event].to_list(), xaxis='x2', yaxis='y2',
                        marker=dict(color=color),
                        name=f"Cluster {event}")
        traces.append(trace)
        
#     # Make traces for graph
#     trace1 = go.Bar(x=teams, y=GFPG, xaxis='x2', yaxis='y2',
#                     marker=dict(color='#0099ff'),
#                     name='Goals For<br>Per Game')
#     trace2 = go.Bar(x=teams, y=GAPG, xaxis='x2', yaxis='y2',
#                     marker=dict(color='#404040'),
#                     name='Goals Against<br>Per Game')

    # Add trace data to figure
    fig.add_traces(traces)

    # initialize xaxis2 and yaxis2
    fig['layout']['xaxis2'] = {}
    fig['layout']['yaxis2'] = {}

    # Edit layout for subplots
    fig.layout.yaxis.update({'domain': [0, .45]})
    fig.layout.yaxis2.update({'domain': [.6, 1]})

    # The graph's yaxis2 MUST BE anchored to the graph's xaxis2 and vice versa
    fig.layout.yaxis2.update({'anchor': 'x2'})
    fig.layout.xaxis2.update({'anchor': 'y2'})
    fig.layout.yaxis2.update({'title': 'Frequency of therapy for cluster'})

    # Update the margins to add a title and see graph x-labels.
    fig.layout.margin.update({'t':75, 'l':50,})
    fig.layout.update({'title': 'Test'})

    # Update the height because adding a graph vertically will interact with
    # the plot height calculated for the table
    fig.layout.update({'height':1000,"width":1000})

    # Plot!
    fig.show()

In [ ]:
# Vertical_Table_and_Chart(events_data[:5])
# Vertical_Table_and_Chart(events_data[5:10])
# Vertical_Table_and_Chart(events_data[10:])

In [ ]:
events=[c for c in df.columns if (':' in c)&(" : " not in c)&("Client" in c)]
events_data=df.groupby("cluster").agg(np.mean)[events]
# events_data=events_data*100

#Делаем красивый порядок колонок
cols=events_data.columns.tolist()
cols.sort()
events_data=events_data[cols]
# events_data=events_data.T
events_data

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

colors_names=["green", "pink", "aqua", "blue", "yellow","black", "red", "coral", "blanchedalmond", "blue",
"blueviolet", "brown", "burlywood", "cadetblue","chartreuse", "chocolate", "coral", "cornflowerblue"]

neq_events_names=[remane_col(ev) for ev in events_data.columns.tolist()]
traces=[]

for i in range(len(events)):
    event=events[i]
    color=colors_names[i]

    fig.add_trace(go.Bar(
        y=[f"Cluster {i}" for i in range(7)],
        x=events_data[event],
        name=neq_events_names[i],
        orientation='h',
        marker=dict(
            color=color,
            line=dict(color=color, width=3)
        )
    ))
# fig.add_trace(go.Bar(
#     y=['giraffes', 'orangutans', 'monkeys'],
#     x=[12, 18, 29],
#     name='LA Zoo',
#     orientation='h',
#     marker=dict(
#         color='rgba(58, 71, 80, 0.6)',
#         line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
#     )
# ))


fig.update_layout(barmode='stack')

fig['layout']['xaxis1'].update(title='Frecuency of intervention for cluster',
                              range=[-0.05, 3.4
                                    ])

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    y=['giraffes', 'orangutans', 'monkeys'],
    x=[20, 14, 23],
    name='SF Zoo',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=['giraffes', 'orangutans', 'monkeys'],
    x=[12, 18, 29],
    name='LA Zoo',
    orientation='h',
    marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))

fig.update_layout(barmode='stack')
fig.show()

In [ ]:
df["T0_age"]=df["T0_age"].astype(float)

    
statistics={}
interventions={}
for clust in range(7):
#     try:
    df_cluster=df[df.cluster==clust]
    goal_features=selected_goal_features(df_cluster,border=5)
    stats_cluster={}
    interventions_cluster={}

    #Количество пациентов в кластере
    stats_cluster['count of patients']=df_cluster.shape[0]

    #Расчитываем %мужчин
    stats_cluster["gender (man), %"]=int(df_cluster["T0_gender : man"].mean()*100)

    #Расчитываем первый поток доверительных интервалов
    for_interval_calculating=['T0_age','duration_treatment']
    
    #Переводим длительность лечения в целое количество месяцев 
    df_cluster["duration_treatment"]=(df_cluster["duration_treatment"]*12).astype(int)

    stats_cluster.update({f"{remane_col(f)}":f"{round(df_cluster.dropna(subset=[f])[f].mean(),1)} {calculate_confidence_interval(df_cluster.dropna(subset=[f])[f])}" for f in for_interval_calculating})


#         #Топ 3 наиболее частых вмешательств
#         (stats_cluster['Top 3 diagnoses in cluster - first'],
#          stats_cluster['Top 3 diagnoses in cluster - second'],
#          stats_cluster['Top 3 diagnoses in cluster - third']
#         )=three_most_frequent_events(df_cluster,type_events="interventions")

    interventions['cluster-specific interventions']=get_specific_interventions(df,clust)

    stats_cluster['cluster-specific diagnoses']=get_specific_interventions(df,clust,type_events="diagnoses")

    #Топ 3 наиболее частых диагнозов
    stats_cluster['top 3 diagnoses in cluster']=three_most_frequent_events(df_cluster,type_events="diagnoses")

    #Рассчитываем средние и ДИ для ЦП
    for_interval_calculating=goal_features

    stats_cluster.update({f"{remane_col(f)}":f"{round(df_cluster.dropna(subset=[f])[f].mean(),1)} {calculate_confidence_interval(df_cluster.dropna(subset=[f])[f])}" for f in for_interval_calculating})

    statistics[f"Cluster #{clust}"]=stats_cluster
#     except:
#         print(f"Падает на кластере {clust} c {df[df.cluster==str(clust)].shape[0]} пациентами")
    
statistics=pd.DataFrame(statistics)
statistics.to_excel("Data/Statistics for clusters.xlsx")
# statistics.dropna()

In [ ]:
df.cluster.value_counts()

In [ ]:
statistics

Основная идея - взять 4-5 кластеров, для увеличения количества людей и повышения адекватности моделей. И используем только последнюю идею - абсолютный прогресс. И завтра у меня созвон в 10. И написать Маркусу

Делаю корреляции для кластеров и вмешательств, пробую Спирмена (использую только T2 и всех пациентов)
Строю БС, и говорю что для каждого набора вмешательств и для конкретного пациента я смогу сделать байесовский вывод, и подобрать оптимальное сочетание вмешательств, и сравнить с линейной регрессией (с помощью ROC-кривых)